## Import libs & metadata

In [2]:
#Import libs 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from tqdm import tqdm
import textwrap
import geopandas as gpd
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse, accuracy_score
from sklearn.metrics.pairwise import euclidean_distances
import random
import mlflow
import mlflow.sklearn
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 15)

In [6]:
#Loading, distribution wells per Platforms,  and modification of metadata.
path = 'C:\\jupyter\\SPP\\input\\'
metadata_init = pd.read_csv(path + 'ACG_wells_metadata.csv', sep=',')
metadata = metadata_init.copy()
metadata = metadata.rename(columns={'X':'X_wellhead', 'Y':'Y_wellhead'})
metadata.Status = metadata.Status.str.strip()
metadata.Status = metadata.Status.str.lower()
metadata.loc[metadata.Status == 'oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'oil producer', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'production', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'produiction oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'production_oil', 'Status' ] = 'production oil'
metadata.loc[metadata.Status == 'abandoned production oil', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'abandoned  oil', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'abandoned oi', 'Status' ] = 'abandoned oil'
metadata.loc[metadata.Status == 'injector  - water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'injector water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'injetor  - water', 'Status' ] = 'injector - water'
metadata.loc[metadata.Status == 'abandoned injector - water per b', 'Status' ] = 'abandoned injector - water'
metadata.loc[metadata.Status == 'plugged and abandoned', 'Status' ] = 'p&a'
metadata.loc[metadata.X_wellhead==118.270, 'X_wellhead'] = 526258.84
metadata.loc[metadata.Y_wellhead==526261.510, 'Y_wellhead'] = 4435802.01
metadata.loc[metadata.well=='C39', 'X_wellhead'] = 526258.840
metadata.loc[metadata.well=='C39', 'Y_wellhead'] = 4435802.010
metadata.loc[metadata.field=='West Azeri', 'field'] = 'WEST AZERI'
metadata.loc[metadata.field=='COP', 'field'] = 'WEST CHIRAG'
metadata.loc[metadata.well=='AZERI2', 'field'] = 'WEST AZERI'
metadata.loc[metadata.well=='AZERI3', 'field'] = 'WEST AZERI'
metadata.loc[metadata.well=='B31', 'field'] = 'CENTRAL AZERI'
metadata.loc[metadata.well=='J28_bpQIP', 'field'] = 'WEST CHIRAG'
field_well_cnt = metadata.groupby('field')['well'].count()
field_well_cnt = field_well_cnt.sort_values(ascending=False).reset_index()
platform = list(field_well_cnt['field'])
figures = list(field_well_cnt['well'])

In [7]:
def well_display(wellname, fmname):
    data = df_prq[(df_prq.wellName==wellname) & (df_prq.FORMATION == fmname)]
    y = data['TST']
    x1 = data['GR_N']
    x2 = data['RHOB'] 
    x2_ = data['NPSS']
    x3 = data['RDEEP']
    x4 = data['PHIT'] 
    x4_ = data['Fluidcode_mod']
    x5 = data['LPERM']
    fig, ax = plt.subplots(1,5, figsize=(7,7), sharey=True)
    ax[0].plot(x1, y, color='lightgreen'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 150)
    ax[1].plot(x2, y, color='red'), ax[1].invert_yaxis(), ax[1].set_xlim(1.65, 2.65)
    twin1 = ax[1].twiny()
    twin1.plot(x2_, y, color='blue')
    twin1.set_xlim(0.6, 0)
    ax[2].plot(x3, y, color='black'), ax[2].set_xscale('log'), ax[2].set_xlim(1, 100), ax[2].invert_yaxis(), ax[2].grid(axis='x', which='both')
    ax[3].plot(x4, y, color='green'), ax[3].set_xlim(0.3, 0), ax[3].grid(axis='x'), ax[3].invert_yaxis()
    twin2 = ax[3].twiny()
    twin2.plot(x4_, y, color='orange', linewidth=0.5)
    twin2.fill_between(x4_, y, color='orange', alpha=0.33)
    twin2.set_xlim(0, 1)
    twin2.set_ylim(min(y), max(y))
    ax[4].plot(x5, y, color='purple'), ax[4].set_xscale('log'), ax[4].set_xlim(1, 1000), ax[4].grid(axis='x'), ax[4].invert_yaxis()
    fig.suptitle(wellname + ' ' + fmname, fontsize=14)
    fig.tight_layout()
def well_display_bal8_10(wellname, num):
    data = df_bal[(df_bal.wellName==wellname) & (df_bal.GR_N > 0)]
    y8 = data[data.FORMATION=='Balakhany VIII sand']['DEPT']
    x1_8 = data[data.FORMATION=='Balakhany VIII sand']['GR_N']
    x2_8 = data[data.FORMATION=='Balakhany VIII sand']['GR_composite_trace']
    y10 = data[data.FORMATION=='Balakhany X sand']['DEPT']
    x1_10 = data[data.FORMATION=='Balakhany X sand']['GR_N']
    x2_10 = data[data.FORMATION=='Balakhany X sand']['GR_composite_trace']  
    fig, ax = plt.subplots(1,2, figsize=(6,7), sharey=False)
    ax[0].plot(x1_8, y8, color='lightgreen', label='GR_N'), ax[0].invert_yaxis(), ax[0].set_xlim(0, 150)
    ax[1].plot(x1_10, y10, color='lightgreen', label='GR_N'), ax[1].invert_yaxis(), ax[1].set_xlim(0, 150)
    twin1 = ax[0].twiny() 
    twin1.plot(x2_8, y8, color='blue', label='GR_cube')
    twin1.set_xlim(0, 150)
    twin2 = ax[1].twiny() 
    twin2.plot(x2_10, y10, color='blue', label='GR_cube')
    twin2.set_xlim(0, 150)
    ax[0].title.set_text('Bal VIII sand')
    ax[1].title.set_text('Bal X sand')
    ax[0].grid()
    ax[1].grid()
    fig.suptitle(str(num+1) + ' ' + wellname, fontsize=14)
    fig.legend(fontsize=8)
    fig.tight_layout()
    # plt.savefig('C:\\jupyter\\SPP\\GR_cube\\wellplots\\' + str(num+1) + '_' + wellname + '.png'
def well_display_auc(wellname):
    data = df_bal[(df_bal.wellName==wellname) & (df_bal.GR_N > 0)]
    delta_rel = data_gb_final_xy_fl[data_gb_final_xy_fl.wellName == wellname]['delta_rel_auc'].iloc[0]
    y8 = data[data.FORMATION=='Balakhany VIII sand']['DEPT']
    x1_8 = data[data.FORMATION=='Balakhany VIII sand']['GR_N']
    x2_8 = data[data.FORMATION=='Balakhany VIII sand']['GR_composite_trace']
    fig, ax = plt.subplots(figsize=(3,7), sharey=False)
    ax.plot(x1_8, y8, color='lightgreen', label='GR_N'), ax.invert_yaxis(), ax.set_xlim(0, 150)
    ax.fill_betweenx(y8, 0, x1_8, color='lightgreen', alpha=.75)
    twin1 = ax.twiny() 
    twin1.plot(x2_8, y8, color='blue', label='GR_cube')
    twin1.set_xlim(0, 150)
    twin1.fill_betweenx(y8, 0, x2_8, color='blue', alpha=.25)
    ax.title.set_text('Bal VIII sand')
    ax.grid()
    fig.suptitle(str(delta_rel) + ' ' + wellname, fontsize=14)
    fig.legend(fontsize=8)
    fig.tight_layout()
    # plt.savefig('C:\\jupyter\\SPP\\GR_cube\\wellplots\\' + str(num+1) + '_' + wellname + '.png')

## GRcube GRN

### Data upload

In [7]:
# Loading the TL well-logs csv file
path = 'C:\\jupyter\\SPP\\input\\'
df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v7.parquet.gzip')

In [4]:
# # Loading the TL well-logs csv file
# path = 'C:\\jupyter\\SPP\\input\\'
# df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v7.parquet.gzip')
# df_prq_fm = df_prq[['wellName','FORMATION', 'X', 'Y']].groupby(['wellName','FORMATION']).apply(lambda x: x.iloc[0]).drop(['wellName','FORMATION'], axis=1)
# # Summarize FLANK data
# well_flanks = df_prq[['wellName','FLANK1','FLANK2']].query("FLANK1 > 0 or FLANK2>0").groupby('wellName').apply(lambda x: x.iloc[0]).drop('wellName',axis=1)
# well_flanks['FLANK'] = well_flanks.FLANK1 + well_flanks.FLANK2
# well_flanks = well_flanks.drop(['FLANK1','FLANK2'], axis=1)
# # Loading the GRcube csv file
# data_init = pd.read_csv(path + 'ACG_GRcube_GRN.csv', sep=',')

In [5]:
# # Data cleaning of TL-dataset
# df = data_init.copy()
# df = df[1:]
# #Select only neccessary data
# df_cln = df[['wellName', 'DEPT', 'FORMATION','GR_composite_trace', 'GR_N']]
# #Fill up nan and -9999 values with 0
# df_cln = df_cln.fillna(0)
# df_cln = df_cln.replace(-9999, 0)
# df_cln = df_cln.replace('-9999', '0')
# #Assign proper types
# dicttypes = {'wellName':'string', 'DEPT':'float', 'FORMATION':'string', 'GR_composite_trace':'float', 'GR_N':'float'}
# df_cln = df_cln.astype(dicttypes, errors='ignore')
# #Select Balakhany zone
# df_bal = df_cln[df_cln.FORMATION.str.contains('Balakhany')]

### AUC data for Bal VIII

In [12]:
# data = df_bal[(df_bal.FORMATION=='Balakhany VIII sand') & (df_bal.GR_N > 0)]
# data['grn_md'] = data.GR_N * 0.1 
# data['grct_md'] = data.GR_composite_trace * 0.1 
# data_gb = data.groupby(['wellName', 'FORMATION'])[['grn_md','grct_md']].sum().reset_index()
# data_gb['delta'] = abs(data_gb.grn_md - data_gb.grct_md)
# data_gb_final = data_gb[(data_gb.grn_md > 0) & (data_gb.grct_md > 0)]
# data_gb_final['delta_rel'] = ((data_gb_final.delta / data_gb_final.grn_md)*100).round(2)
# data_gb_final_xy = data_gb_final.set_index(['wellName', 'FORMATION']).join(df_prq_fm).reset_index()
# data_gb_final_xy_fl = data_gb_final_xy.set_index('wellName').join(well_flanks).reset_index()
# data_gb_final_xy_fl.rename(columns={'delta':'delta_auc', 'delta_rel':'delta_rel_auc'}, inplace=True)
# data_gb_final_xy_fl.sort_values('delta_rel_auc').head()
# data_auc8 = data_gb_final_xy_fl

In [6]:
# # Comparison AUC GR-cube vs GR-log
# fig = go.Figure()
# fig.add_trace(go.Histogram(x=data_gb_final_xy_fl.delta_rel_auc, xbins=dict(start=0, end=100.0, size=5)))
# fig.update_layout(title_text='Comparison AUC GR-cube vs GR-log',xaxis_title_text='delta_rel', yaxis_title_text='Count',
#                   autosize=True, width=1200, height=300, margin=dict(l=10,r=10,b=10,t=40))

In [8]:
# fig = go.Figure()
# field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
# fig.add_trace(go.Scatter(x=data_gb_final_xy_fl.X, y=data_gb_final_xy_fl.Y, customdata = data_gb_final_xy_fl[['wellName', 'delta_auc','FLANK']],
#                          marker=dict(color=data_gb_final_xy_fl.FLANK, size=data_gb_final_xy_fl.delta_rel_auc, line=dict(color='rgb(47, 57, 61)', width=1)),
#                          mode='markers', name='well delta_rel',
#                          hovertemplate="".join(["well:%{customdata[0]}, delta_rel:%{customdata[1]}<extra></extra>"])))
# fig.add_trace(go.Scatter(x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
#                          text=field_avg_coord['field'], textposition="middle right",
#                          marker=dict(color='rgb(0, 0,0)', size=12),
#                          mode='markers+text', name='Platforms', 
#                          marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])))
# fig.update_layout(title_text='Balakhany VIII sand delta_rel (%) AUC, color=f(FLANK)',
#                   autosize=True, width=1200, height=700, margin=dict(l=10,r=10,b=10,t=40))
# fig.update_layout(legend=dict( yanchor="top", y=1, xanchor="right", x=1, bgcolor='rgba(255,255,255,1)', bordercolor='Black',borderwidth=1))
# fig.show()

In [9]:
# well_display_auc('F02')

In [200]:
# delta_rel10  = data_gb_final_xy_fl[data_gb_final_xy_fl.delta_rel > 10]
# for n, well in enumerate(delta_rel10.wellName.unique()):
#     well_display_onr(wellname=well)

### AUC data for Bal X

In [11]:
# data = df_bal[(df_bal.FORMATION=='Balakhany X sand') & (df_bal.GR_N > 0)]
# data['grn_md'] = data.GR_N * 0.1 
# data['grct_md'] = data.GR_composite_trace * 0.1 
# data_gb = data.groupby(['wellName', 'FORMATION'])[['grn_md','grct_md']].sum().reset_index()
# data_gb['delta'] = abs(data_gb.grn_md - data_gb.grct_md)
# data_gb_final = data_gb[(data_gb.grn_md > 0) & (data_gb.grct_md > 0)]
# data_gb_final['delta_rel'] = ((data_gb_final.delta / data_gb_final.grn_md)*100).round(2)
# data_gb_final_xy = data_gb_final.set_index(['wellName', 'FORMATION']).join(df_prq_fm).reset_index()
# data_gb_final_xy_fl = data_gb_final_xy.set_index('wellName').join(well_flanks).reset_index()
# data_gb_final_xy_fl.rename(columns={'delta':'delta_auc', 'delta_rel':'delta_rel_auc'}, inplace=True)
# data_gb_final_xy_fl.sort_values('delta_rel_auc').head()
# data_auc10 = data_gb_final_xy_fl

## Data cleaning/verification VSHcube & VSHlogs

### Working with unite Bal VIII / X

In [8]:
df_bal_net2_kh = pd.read_csv('C:\\jupyter\\SPP\\inputoutput\\df_bal_net2_kh_save.csv')
df_bal_net2_kh = df_bal_net2_kh.dropna()
df_bal_net2_kh['net_tst'] = df_bal_net2_kh.TST_smpl * df_bal_net2_kh.NET
df_bal_net2_kh['net_clp_tst'] = df_bal_net2_kh.TST_smpl * df_bal_net2_kh.NET_clp
df_bal_net2_kh['net_clp2_tst'] = df_bal_net2_kh.TST_smpl * df_bal_net2_kh.NET_clp2
df_bal_net2_kh_gb = df_bal_net2_kh.groupby(['well','FORMATION_up'])[['net_tst','net_clp_tst','net_clp2_tst']].sum().reset_index()

In [ ]:
path = 'C:\\jupyter\\SPP\\input\\'
vsh_cube_log = pd.read_parquet(path + 'ACG_GRcube_VSH_v3.parquet.gzip')
vsh_cube_log

In [ ]:
vsh_cube_log

In [38]:
path = 'C:\\jupyter\\SPP\\input\\'
vsh_cube_log = pd.read_parquet(path + 'ACG_GRcube_VSH_v3.parquet.gzip')
vsh_cube_log = vsh_cube_log.replace(-9999.000, np.nan)
vsh_cube_log = vsh_cube_log.dropna()
vsh_cube_log.loc[vsh_cube_log.FORMATION.str.contains('Balakhany VIII'), 'FORMATION_up'] = 'Balakhany VIII'
vsh_cube_log.loc[vsh_cube_log.FORMATION.str.contains('Balakhany X'), 'FORMATION_up'] = 'Balakhany X'
vsh_cube_log = vsh_cube_log[vsh_cube_log.FORMATION_up.isin(['Balakhany VIII', 'Balakhany X'])]

smooth = 25
vsh_cube_log['VSH_GRlog_smooth'] = vsh_cube_log['VSH_GRlog'].rolling(window=smooth, center=True).mean()
vsh_cube_log = vsh_cube_log.fillna(0)

In [ ]:
def vsh_net_calculation(dataset, vsh_log):
    dataset.loc[dataset['VSH_GRcube'] < 0.5, 'vsh_net_cube'] =1
    dataset.loc[dataset['VSH_GRcube'] >= 0.5, 'vsh_net_cube'] =0
    dataset.loc[dataset[vsh_log] < 0.5, 'vsh_net_log'] =1
    dataset.loc[dataset[vsh_log] >= 0.5, 'vsh_net_log'] =0
    return dataset
vsh_cube_log = vsh_net_calculation(vsh_cube_log, 'VSH_GRlog_smooth')
def net_sum_calculation(dataset):
    dataset['TST_sample'] = dataset.TST.diff()
    dataset['net_cube_tst'] = dataset.TST_sample * dataset.vsh_net_cube
    dataset['net_log_tst'] = dataset.TST_sample * dataset.vsh_net_log
    dataset = dataset.dropna()
    dataset_v2 = dataset.groupby(['wellName','FORMATION_up'])[['DEPT','TST_sample']].apply(lambda x: x.iloc[0]).reset_index()
    dataset_v2['TST_sample_remove'] = 1
    dataset_v3 = dataset.set_index(['wellName', 'FORMATION_up','DEPT']).join(
                        dataset_v2.set_index(['wellName', 'FORMATION_up','DEPT']), rsuffix='_r').reset_index()
    dataset_v3 = dataset_v3[dataset_v3.TST_sample_remove != 1]
    result = dataset_v3.groupby(['wellName','FORMATION_up'])[['net_cube_tst','net_log_tst']].sum().reset_index()
    return result
res_net_cube_logs = net_sum_calculation(vsh_cube_log)
def net_data_processing(dataset):
    dataset = dataset.replace('na', np.nan)
    dataset = dataset.dropna()
    dict_var = {'wellName':'string',
                'FORMATION_up':'string',
                'net_cube_tst':float,
                'net_log_tst':float}
    dataset = dataset.astype(dict_var)
    dataset['diff'] = (dataset.net_cube_tst - dataset.net_log_tst).round(2) 
    dataset['diff_rel'] = (abs(dataset.net_cube_tst - dataset.net_log_tst)/dataset.net_log_tst).round(2)
    dataset['qc_status'] = 'green'
    dataset.loc[dataset.diff_rel >= 0.33, 'qc_status'] = 'yellow'
    dataset.loc[dataset.diff_rel >= 0.66, 'qc_status'] = 'red'
    return dataset
res_net_cube_logs = net_data_processing(res_net_cube_logs)
res_net_cube_logs = res_net_cube_logs.rename(columns={'wellName':'well'})
# Calculate qc distribution
print(f'classification qc to 0.33 - 0.66 - 1 bins with roll_avg {smooth}:\r\n', res_net_cube_logs['qc_status'].value_counts(normalize=True))
# Analize of red-quality data
res_net_cube_logs_red = res_net_cube_logs[res_net_cube_logs.qc_status == 'red']
res_net_cube_logs_red = res_net_cube_logs_red.sort_values(by=['FORMATION_up', 'well'])
res_net_cube_logs_green = res_net_cube_logs[res_net_cube_logs.qc_status == 'green']
res_net_cube_logs_green = res_net_cube_logs_green.sort_values(by=['FORMATION_up', 'well'])
res_net_cube_logs_net_clp = res_net_cube_logs.set_index(['well','FORMATION_up']).join(
                            df_bal_net2_kh_gb.set_index(['well','FORMATION_up'])).reset_index()
res_net_cube_logs_net_clp = res_net_cube_logs_net_clp.dropna()
res_net_cube_logs_net_clp['diff_net2'] = res_net_cube_logs_net_clp.net_cube_tst - res_net_cube_logs_net_clp.net_clp2_tst
res_net_cube_logs_net_clp = res_net_cube_logs_net_clp[res_net_cube_logs_net_clp.net_clp2_tst > 0]  

In [ ]:
def resid_analize_plot(data, max_val, y_lim, text):
    data = data.round({'net_log_tst':0, 'net_cube_tst':0, 'diff':2, 'net_clp2_tst':0, 'diff_net2':2})
    points_cube_log = go.Scatter(   x=data['net_log_tst'], y=data['diff'],
                                    mode='markers',
                                    marker=dict(color='blue', size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                    customdata = data[['well', 'net_cube_tst', 'net_log_tst', 'diff', 'FORMATION_up']],
                                    hovertemplate="".join(
                                    [   " w: %{customdata[0]}",
                                        " net_cube: %{customdata[1]}", 
                                        " net_log: %{customdata[2]}", 
                                        " diff: %{customdata[3]}", 
                                        " f: %{customdata[4]}<extra></extra>"]), name = 'cube_log_diff')
    points_cube_net2 = go.Scatter(  x=data['net_log_tst'], y=data['diff_net2'],
                                    mode='markers',
                                    marker=dict(color='red', size=7, opacity=0.75, line=dict(color='rgb(47, 57, 61)', width=0.5)),
                                    customdata = data[['well','net_cube_tst', 'net_clp2_tst', 'diff_net2', 'FORMATION_up']],
                                    hovertemplate="".join(
                                    [   " w: %{customdata[0]}",
                                        " net_cube: %{customdata[1]}",
                                        " net_clp2: %{customdata[2]}", 
                                        " diff_net2: %{customdata[3]}", 
                                        " f: %{customdata[4]}<extra></extra>"]),  name = 'cube_net2_diff')
    fig = go.Figure()
    line = go.Scatter(x=[0, max_val], y=[0, 0], mode='lines+markers', line=dict(color='black'),  name = 'zero line')
    fig.add_trace(points_cube_log)
    fig.add_trace(points_cube_net2)
    fig.add_trace(line)
    fig.update_xaxes(   title_text='net_log_tst')
    fig.update_yaxes(   title_text='diff')
    fig.update_layout(  title_text= ('Residuals analize' + ' ' + text), width=800, height=500,
                        xaxis=dict(range=[0, max_val]), yaxis=dict(range=[-1*(y_lim), y_lim]),
                        margin=dict(l=10,r=10,b=10,t=50), showlegend=True)
    return fig.show()
resid_analize_plot(res_net_cube_logs_net_clp, 250, 75, 'diff = net_cube_tst - net_log_tst')

In [ ]:
def well_display_vsh(dataset, dataset_with_sums, wellname, fmname):
    fig, ax = plt.subplots(1,2, figsize=(6,6), sharey=False)
    sums = dataset_with_sums[(dataset_with_sums.well == wellname) & (dataset_with_sums.FORMATION_up == fmname)]
    net_log_tst = sums['net_log_tst'].iloc[0].round(1)
    net_cube_tst = sums['net_cube_tst'].iloc[0].round(1)
    diff_rel = sums['diff_rel'].iloc[0].round(2)
    data = dataset[(dataset.wellName == wellname) & (dataset.FORMATION_up == fmname)]
    dept = data['TST']
    vsh_cube = data['VSH_GRcube']
    vsh_log = data['VSH_GRlog']
    vsh_log_smooth = data['VSH_GRlog_smooth']
    net_cube = data['vsh_net_cube']
    net_log = data['vsh_net_log'] 
    ax[0].plot(vsh_cube, dept, color='green') 
    ax[0].set_xlim(0, 1)
    twin2 = ax[0].twiny()
    twin2.plot(net_cube, dept, color='orange', alpha=.75, linewidth=1)
    twin2.fill_betweenx(dept, net_cube, 0,  color='orange', alpha=0.33)
    twin2.set_ylim(max(dept), min(dept))
    twin2.set_xlim(0, 2)
    ax[0].set_title('GR_cube NET: ' + str(net_cube_tst) + ' m', fontsize=12)    
    ax[1].plot(vsh_log, dept, color='green', linewidth=2, alpha=0.66)
    ax[1].plot(vsh_log_smooth, dept, color='red', linewidth=1) 
    ax[1].set_xlim(0, 1)
    twin3 = ax[1].twiny()
    twin3.plot(net_log, dept, color='orange', alpha=.75, linewidth=1)
    twin3.fill_betweenx(dept, net_log, 0, color='orange', alpha=0.33)
    twin3.set_ylim(max(dept), min(dept))
    twin3.set_xlim(0, 2)
    ax[1].set_title('GR_log NET: ' + str(net_log_tst) + ' m',  fontsize=12)   
    fig.suptitle(wellname + ' ' + fmname + ' ' + 'diff_rel:' + str(diff_rel), fontsize=14)
    fig.tight_layout()
for ind,row in res_net_cube_logs_red[:].iterrows():
    well_display_vsh(vsh_cube_log, res_net_cube_logs_red, row['well'], row['FORMATION_up'])

In [ ]:
res_net_cube_logs_green[12:15]

In [ ]:
for ind,row in res_net_cube_logs_green[12:15].iterrows():
    well_display_vsh(vsh_cube_log, res_net_cube_logs_green, row['well'], row['FORMATION_up'])

### Working with FU

In [13]:
# Uploading GRcube
path = 'C:\\jupyter\\SPP\\input\\'
datavsh_init = pd.read_csv(path + 'ACG_GRcube_VSH_v2.csv', sep=',')
datavsh_init['wellName'] = datavsh_init.wellName.astype('string')
datavsh_init['datasetName'] = datavsh_init.datasetName.astype('string')
datavsh_init['FORMATION'] = datavsh_init.FORMATION.astype('string')
datavsh = datavsh_init.copy()
# Control of avalaiblity all data for comparison Vshcube vs Vshlog 
datavsh = datavsh_init.copy()
vshcube = datavsh[['wellName', 'DEPT', 'FORMATION','VSH_GRcube']]
df_prq_vsh = df_prq[['wellName', 'DEPTH', 'FORMATION','TST', 'VSH', 'GR_N']]
df_prq_vsh_bal = df_prq_vsh[(df_prq_vsh.FORMATION.isin(['Balakhany VIII sand', 'Balakhany X sand']))]
vshlog = df_prq_vsh_bal[['wellName', 'DEPTH', 'FORMATION', 'VSH','GR_N']]
vshlog.rename(columns={'DEPTH':'DEPT', 'VSH':'VSH_GRlog'},inplace=True)
vsh_final = vshcube.set_index(['wellName', 'DEPT']).join(vshlog.set_index(['wellName', 'DEPT']), rsuffix = '_log').reset_index()
vsh_final.rename(columns={'FORMATION':'FORMATION_cube'}, inplace=True)
vsh_final['VSH_GRlog'] = vsh_final.VSH_GRlog.rolling(50, min_periods=10, center=True, win_type='gaussian').mean(std=5)
vsh_final.loc[vsh_final.VSH_GRcube < 0.5, 'gross_net_cb'] =1
vsh_final.loc[vsh_final.VSH_GRcube >= 0.5, 'gross_net_cb'] =0
vsh_final.loc[vsh_final.VSH_GRlog < 0.5, 'gross_net_lg'] =1
vsh_final.loc[vsh_final.VSH_GRlog >= 0.5, 'gross_net_lg'] =0
# Counting how many data points for VSHcube vs VSHlog
vsh_final_cube8 = vsh_final.groupby(['wellName','FORMATION_cube'])['VSH_GRcube'].count()
vsh_final_log8 = vsh_final.groupby(['wellName','FORMATION_cube'])['VSH_GRlog'].count()
df_vsh_cube8 = pd.DataFrame(vsh_final_cube8).reset_index()
df_vsh_cube8.rename(columns={'FORMATION_cube':'FORMATION'}, inplace=True)
df_vsh_logs8 = pd.DataFrame(vsh_final_log8).reset_index()
df_vsh_logs8.rename(columns={'FORMATION_cube':'FORMATION'}, inplace=True)
df_vsh_vshs8 = df_vsh_cube8.set_index(['wellName','FORMATION']).join(df_vsh_logs8.set_index(['wellName','FORMATION'])).reset_index()
df_vsh_vshs8['delta_cube_logs'] = ((df_vsh_vshs8.VSH_GRcube - df_vsh_vshs8.VSH_GRlog)/df_vsh_vshs8.VSH_GRcube).round(2)
# # Checking wells above and below 25% of delta_rel
# below25_bal8 = gross_net_rel_fld_xy[(gross_net_rel_fld_xy.delta_rel < 0.25) & 
#                                    (gross_net_rel_fld_xy.FORMATION == 'Balakhany VIII sand')]
# above25_bal8 = gross_net_rel_fld_xy[(gross_net_rel_fld_xy.delta_rel >= 0.25) & 
#                                    (gross_net_rel_fld_xy.FORMATION == 'Balakhany VIII sand')]
# # Final well problems lists
# well_below25_bal8 = below25_bal8.wellName.unique()
# well_above25_bal8 = above25_bal8.wellName.unique()
wells_broken_vsh_log = df_vsh_vshs8[df_vsh_vshs8.delta_cube_logs > 0.25].wellName.unique()

In [203]:
# df_vsh_vshs8[df_vsh_vshs8.wellName.isin(wells_broken_vsh_log) & 
#             (df_vsh_vshs8.FORMATION == 'Balakhany VIII sand')].sort_values(by='delta_cube_logs')

## GRcube VSH

#### Functions

In [204]:
# Function to display well plots with logging curves
def well_display_vsh_gr8(wellname):
    fig, ax = plt.subplots(1,3, figsize=(6,6), sharey=False)
    bal8 = vsh_final[(vsh_final.wellName==wellname) & (vsh_final.FORMATION_cube == 'Balakhany VIII sand')]
    dept8 = bal8['DEPT']
    vsh_c8 = bal8['VSH_GRcube']
    vsh_l8 = bal8['VSH_GRlog']
    gr_l8 = bal8['GR_N']
    gsn_c8 = bal8['gross_net_cb']
    gsn_l8 = bal8['gross_net_lg'] 
    ax[0].plot(vsh_c8, dept8, color='green') 
    ax[0].fill_betweenx(dept8,0, vsh_c8,color='green', alpha=0.25)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 1)
    twin0 = ax[0].twiny()
    twin0.plot(gsn_c8, dept8, color='orange', alpha=.75, linewidth=1)
    twin0.fill_betweenx(dept8,0,gsn_c8, color='orange', alpha=0.5)
    twin0.set_xlim(0, 1)
    twin0.set_ylim(max(dept8), min(dept8))  
    ax[1].plot(gr_l8, dept8, color='green')
    ax[1].invert_yaxis() 
    ax[1].set_xlim(0, 150)
    ax[2].plot(vsh_l8, dept8, color='green')
    ax[2].fill_betweenx(dept8,0, vsh_l8,color='green', alpha=0.25) 
    ax[2].invert_yaxis() 
    ax[2].set_xlim(0, 1)
    twin1 = ax[2].twiny()
    twin1.plot(gsn_l8, dept8, color='orange', alpha=.75, linewidth=1)
    twin1.fill_betweenx(dept8, 0, gsn_l8, color='orange', alpha=0.5)
    twin1.set_xlim(0, 1) 
    twin1.set_ylim(max(dept8), min(dept8))
    fig.suptitle(wellname, fontsize=14)
    fig.tight_layout()
def well_display_vsh_gr10(wellname):
    fig, ax = plt.subplots(1,3, figsize=(6,6), sharey=False)
    bal8 = vsh_final[(vsh_final.wellName==wellname) & (vsh_final.FORMATION_cube == 'Balakhany X sand')]
    dept8 = bal8['DEPT']
    vsh_c8 = bal8['VSH_GRcube']
    vsh_l8 = bal8['VSH_GRlog']
    gr_l8 = bal8['GR_N']
    gsn_c8 = bal8['gross_net_cb']
    gsn_l8 = bal8['gross_net_lg'] 
    ax[0].plot(vsh_c8, dept8, color='green') 
    ax[0].fill_betweenx(dept8,0, vsh_c8,color='green', alpha=0.25)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 1)
    twin0 = ax[0].twiny()
    twin0.plot(gsn_c8, dept8, color='orange', alpha=.75, linewidth=1)
    twin0.fill_betweenx(dept8,0,gsn_c8, color='orange', alpha=0.5)
    twin0.set_xlim(0, 1)
    twin0.set_ylim(max(dept8), min(dept8))  
    ax[1].plot(gr_l8, dept8, color='green')
    ax[1].invert_yaxis() 
    ax[1].set_xlim(0, 150)
    ax[2].plot(vsh_l8, dept8, color='green')
    ax[2].fill_betweenx(dept8,0, vsh_l8,color='green', alpha=0.25) 
    ax[2].invert_yaxis() 
    ax[2].set_xlim(0, 1)
    twin1 = ax[2].twiny()
    twin1.plot(gsn_l8, dept8, color='orange', alpha=.75, linewidth=1)
    twin1.fill_betweenx(dept8, 0, gsn_l8, color='orange', alpha=0.5)
    twin1.set_xlim(0, 1) 
    twin1.set_ylim(max(dept8), min(dept8))
    fig.suptitle(wellname, fontsize=14)
    fig.tight_layout()
def well_display_vsh8(wellname):
    fig, ax = plt.subplots(1,2, figsize=(6,6), sharey=False)
    bal8 = vsh_final[(vsh_final.wellName==wellname) & (vsh_final.FORMATION_cube == 'Balakhany VIII sand')]
    dept8 = bal8['DEPT']
    gr_c8 = bal8['VSH_GRcube']
    gr_l8 = bal8['VSH_GRlog'] 
    gsn_c8 = bal8['gross_net_cb']
    gsn_l8 = bal8['gross_net_lg'] 
    ax[0].plot(gr_c8, dept8, color='green')
    ax[0].fill_betweenx(dept8,0, gr_c8,color='green', alpha=0.25)
    ax[0].invert_yaxis() 
    ax[0].set_xlim(0, 1)
    twin0 = ax[0].twiny()
    twin0.plot(gsn_c8, dept8, color='orange', alpha=.75, linewidth=1)
    twin0.fill_betweenx(dept8,0,gsn_c8,  color='orange', alpha=0.5)
    twin0.set_xlim(0, 1)
    twin0.set_ylim(max(dept8), min(dept8))  
    ax[1].plot(gr_l8, dept8, color='green')
    ax[1].fill_betweenx(dept8,0, gr_l8,color='green', alpha=0.25) 
    ax[1].invert_yaxis() 
    ax[1].set_xlim(0, 1)
    twin1 = ax[1].twiny()
    twin1.plot(gsn_l8, dept8, color='orange', alpha=.75, linewidth=1)
    twin1.fill_betweenx(dept8, 0, gsn_l8, color='orange', alpha=0.5)
    twin1.set_xlim(0, 1) 
    twin1.set_ylim(max(dept8), min(dept8))
    fig.suptitle(wellname, fontsize=14)
    fig.tight_layout()
def well_display_vsh10(wellname):
    fig, ax = plt.subplots(1,2, figsize=(6,6), sharey=False)
    bal10 = vsh_final[(vsh_final.wellName==wellname) & (vsh_final.FORMATION_cube == 'Balakhany X sand')]
    dept10 = bal10['DEPT']
    gr_c10 = bal10['VSH_GRcube']
    gr_l10 = bal10['VSH_GRlog'] 
    gsn_c10 = bal10['gross_net_cb']
    gsn_l10 = bal10['gross_net_lg'] 
    ax[0].plot(gr_c10, dept10, color='green') 
    # ax[1,0].invert_yaxis() 
    ax[0].set_xlim(0, 1)
    twin2 = ax[0].twiny()
    twin2.plot(gsn_c10, dept10, color='orange', alpha=.75, linewidth=2)
    twin2.fill_between(dept10, gsn_c10, color='orange', alpha=0.33)
    twin2.set_ylim(max(dept10), min(dept10))
    twin2.set_xlim(0, 2)    
    ax[1].plot(gr_l10, dept10, color='green') 
    # ax[1,1].invert_yaxis() 
    ax[1].set_xlim(0, 1)
    twin3 = ax[1].twiny()
    twin3.plot(gsn_l10, dept10, color='orange', alpha=.75, linewidth=2)
    twin3.fill_between(dept10, gsn_l10, color='orange', alpha=0.33)
    twin3.set_ylim(max(dept10), min(dept10))
    twin3.set_xlim(0, 2)   
    fig.suptitle(wellname, fontsize=14)
    fig.tight_layout()
def well_sheet_display(well, code=1):
    """
    very needed here the df !gross_net_rel_fld_xy!
    """
    if code == 1:
        result = gross_net_rel_fld_xy[(gross_net_rel_fld_xy.wellName == well) &
                                      (gross_net_rel_fld_xy.FORMATION == 'Balakhany VIII sand')][['wellName','FORMATION','gross_net_cb','gross_net_lg','delta_rel_vsh']]
    if code == 2:
        result = gross_net_rel_fld_xy[(gross_net_rel_fld_xy.wellName == well) &
                                      (gross_net_rel_fld_xy.FORMATION == 'Balakhany X sand')][['wellName','FORMATION','gross_net_cb','gross_net_lg','delta_rel_vsh']]
    return result

#### Comparison VSHcube vs VSHlogs

In [ ]:
# Uploading GRcube
path = 'C:\\jupyter\\SPP\\input\\'
datavsh_init = pd.read_csv(path + 'ACG_GRcube_VSH_v2.csv', sep=',')
datavsh_init['wellName'] = datavsh_init.wellName.astype('string')
datavsh_init['datasetName'] = datavsh_init.datasetName.astype('string')
datavsh_init['FORMATION'] = datavsh_init.FORMATION.astype('string')
datavsh = datavsh_init.copy()
# Calculation gross_net for VSHcube 
datavsh.loc[datavsh.VSH_GRcube < 0.5, 'gross_net_cb'] =1
datavsh.loc[datavsh.VSH_GRcube >= 0.5, 'gross_net_cb'] =0
# Counting gross_net_cube points for whole formations 
gr_cube_gross_net = datavsh.groupby(['wellName','FORMATION'])['gross_net_cb'].sum().reset_index()
gr_cube_gross_net['gross_net_cb'] = gr_cube_gross_net.gross_net_cb*0.1
# Preparation data for gross_net VSHlogs
df_prq_vsh = df_prq[['wellName', 'DEPTH', 'FORMATION','TST', 'VSH']]
df_prq_vsh_bal = df_prq_vsh[(df_prq_vsh.FORMATION.isin(['Balakhany VIII sand', 'Balakhany X sand'])) &
                             (df_prq_vsh.VSH > 0)]
# Calculation gross_net for VSHlogs                        
df_prq_vsh_bal.loc[df_prq_vsh_bal.VSH < 0.5, 'gross_net_lg'] =1
df_prq_vsh_bal.loc[df_prq_vsh_bal.VSH >= 0.5, 'gross_net_lg'] =0
# Counting gross_net_logs points for whole formations   
gr_log_gross_net = df_prq_vsh_bal.groupby(['wellName','FORMATION'])['gross_net_lg'].sum().reset_index()
gr_log_gross_net['gross_net_lg'] = gr_log_gross_net.gross_net_lg*0.1
# Preparatio data for delta_rel for VSHcube vs VSHlogs taking into account "wells_broken_vsh_log"
gr_cube_gross_net_total = gr_cube_gross_net.set_index(['wellName', 'FORMATION']).join(gr_log_gross_net.set_index(['wellName', 'FORMATION'])).reset_index()
gr_cube_gross_net_total2 = gr_cube_gross_net_total[~gr_cube_gross_net_total.wellName.isin(wells_broken_vsh_log)]
# Correction gross_net_lg for wrong depth sample rate
sample_problem = vshlog.groupby('wellName')['DEPT'].apply(lambda x: x.iloc[101]-x.iloc[100]).reset_index()
sample_problem0_5 = sample_problem[(sample_problem.DEPT >= 0.5)].wellName
sample_problem0_1524 = sample_problem[(sample_problem.DEPT >= 0.15) & (sample_problem.DEPT < 0.18)].wellName
sample_problem0_2 = sample_problem[(sample_problem.DEPT >= 0.18) & (sample_problem.DEPT < 0.5)].wellName
gr_cube_gross_net_total2.loc[gr_cube_gross_net_total2.wellName.isin(sample_problem0_5), 'gross_net_lg'] = gr_cube_gross_net_total2.gross_net_lg*5
gr_cube_gross_net_total2.loc[gr_cube_gross_net_total2.wellName.isin(sample_problem0_2), 'gross_net_lg'] = gr_cube_gross_net_total2.gross_net_lg*2
gr_cube_gross_net_total2.loc[gr_cube_gross_net_total2.wellName.isin(sample_problem0_1524), 'gross_net_lg'] = gr_cube_gross_net_total2.gross_net_lg*1.524
# Calcaulation delta & delta_rel - MAIN POINT
gr_cube_gross_net_total2['delta'] = abs(gr_cube_gross_net_total2.gross_net_lg - gr_cube_gross_net_total2.gross_net_cb)
gr_cube_gross_net_total2['delta_rel'] = (gr_cube_gross_net_total2.delta/gr_cube_gross_net_total2.gross_net_lg).round(2)
gross_net_rel = gr_cube_gross_net_total2.copy()
# Joining metadata to delta_rel dataframe
gross_net_rel_fld = gross_net_rel.set_index('wellName').join(metadata[['well','field']].set_index('well')).reset_index()
gross_net_rel_fld.rename(columns={'index':'wellName'}, inplace=True)
gross_net_rel_fld_xy = gross_net_rel_fld.set_index(['wellName', 'FORMATION']).join(df_prq_fm).reset_index()
gross_net_rel_fld_xy['field'] = gross_net_rel_fld_xy.field.astype('string')
# Assigning numerical values insted text names
gross_net_rel_fld_xy.loc[gross_net_rel_fld_xy.field == 'DDGG', 'field_num'] = 1
gross_net_rel_fld_xy.loc[gross_net_rel_fld_xy.field == 'DWG', 'field_num'] = 2
gross_net_rel_fld_xy.loc[gross_net_rel_fld_xy.field == 'WEST CHIRAG', 'field_num'] = 3
gross_net_rel_fld_xy.loc[gross_net_rel_fld_xy.field == 'CHIRAG', 'field_num'] = 4
gross_net_rel_fld_xy.loc[gross_net_rel_fld_xy.field == 'WEST AZERI', 'field_num'] = 5
gross_net_rel_fld_xy.loc[gross_net_rel_fld_xy.field == 'CENTRAL AZERI', 'field_num'] = 6
gross_net_rel_fld_xy.loc[gross_net_rel_fld_xy.field == 'EAST AZERI', 'field_num'] = 7
# Assigning categorical marks to qc_gross_net
gross_net_rel_fld_xy.loc[gross_net_rel_fld_xy.delta_rel <=0.33, 'qc_gross_net'] = 1
gross_net_rel_fld_xy.loc[(gross_net_rel_fld_xy.delta_rel >0.33) &
                           (gross_net_rel_fld_xy.delta_rel <=0.66), 'qc_gross_net'] = 2
gross_net_rel_fld_xy.loc[(gross_net_rel_fld_xy.delta_rel >0.66),'qc_gross_net'] = 3
# Assigning categorical marks to qc_gross_net_v2
gross_net_rel_fld_xy.loc[gross_net_rel_fld_xy.delta_rel <=0.25, 'qc_gross_net_v2'] = 1
gross_net_rel_fld_xy.loc[(gross_net_rel_fld_xy.delta_rel >0.25) &
                           (gross_net_rel_fld_xy.delta_rel <=0.50), 'qc_gross_net_v2'] = 2
gross_net_rel_fld_xy.loc[(gross_net_rel_fld_xy.delta_rel >0.50) &
                           (gross_net_rel_fld_xy.delta_rel <=0.75), 'qc_gross_net_v2'] = 3
gross_net_rel_fld_xy.loc[(gross_net_rel_fld_xy.delta_rel >0.75),'qc_gross_net_v2'] = 4
data_vsh_drel = gross_net_rel_fld_xy

In [ ]:
# Balakhany VIII sand delta_rel 3-colors Hmd GRcube vs GRlog
fig = go.Figure()
field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
gross_net_rel_fld_xy_8 =gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION=='Balakhany VIII sand') & 
                                             (gross_net_rel_fld_xy.delta_rel.notna()) &
                                             (gross_net_rel_fld_xy.delta_rel <=1)]
gross_net_rel_fld_xy_10 =gross_net_rel_fld_xy[gross_net_rel_fld_xy.FORMATION=='Balakhany X sand']
color_scale = ('#02bd46', '#ffb303', '#d40404')
fig.add_trace(go.Scatter(x=gross_net_rel_fld_xy_8.X, y=gross_net_rel_fld_xy_8.Y, customdata = gross_net_rel_fld_xy_8[['wellName', 'delta_rel','field']],
                         marker=dict(color=gross_net_rel_fld_xy_8.qc_gross_net, size=10, 
                                     line=dict(color='rgb(47, 57, 61)', width=1), colorscale=color_scale),
                         mode='markers', name='vshc/vshl delta_rel',
                         hovertemplate="".join(["well:%{customdata[0]}, delta_rel:%{customdata[1]}<extra></extra>"])))
fig.add_trace(go.Scatter(x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
                         text=field_avg_coord['field'], textposition="middle right",
                         marker=dict(color='rgb(0, 0,0)', size=12),
                         mode='markers+text', name='Platforms', 
                         marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])))
fig.update_layout(title_text='Balakhany VIII sand delta_rel (v/v) Hmd GRcube vs GRlog, color=f(platform)',
                  autosize=True, width=1200, height=700, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(legend=dict( yanchor="top", y=1, xanchor="right", x=1, bgcolor='rgba(255,255,255,1)', bordercolor='Black',borderwidth=1))
fig.show()

In [ ]:
# Balakhany VIII sand delta_rel 4-colors_v2 Hmd GRcube vs GRlog
fig = go.Figure()
field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
gross_net_rel_fld_xy_8 =gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION=='Balakhany VIII sand') & 
                                             (gross_net_rel_fld_xy.delta_rel.notna()) &
                                             (gross_net_rel_fld_xy.delta_rel <=1)]
gross_net_rel_fld_xy_10 =gross_net_rel_fld_xy[gross_net_rel_fld_xy.FORMATION=='Balakhany X sand']
color_scale = ('#02bd46', '#ffb303', '#d40404', '#827d7d')
fig.add_trace(go.Scatter(x=gross_net_rel_fld_xy_8.X, y=gross_net_rel_fld_xy_8.Y, customdata = gross_net_rel_fld_xy_8[['wellName', 'delta_rel','field']],
                         marker=dict(color=gross_net_rel_fld_xy_8.qc_gross_net_v2, size=10, 
                                     line=dict(color='rgb(47, 57, 61)', width=1), colorscale=color_scale),
                         mode='markers', name='vshc/vshl delta_rel',
                         hovertemplate="".join(["well:%{customdata[0]}, delta_rel:%{customdata[1]}<extra></extra>"])))
fig.add_trace(go.Scatter(x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
                         text=field_avg_coord['field'], textposition="middle right",
                         marker=dict(color='rgb(0, 0,0)', size=12),
                         mode='markers+text', name='Platforms', 
                         marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])))
fig.update_layout(title_text='Balakhany VIII sand delta_rel (v/v) Hmd GRcube vs GRlog, color=f(platform)',
                  autosize=True, width=1200, height=700, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(legend=dict( yanchor="top", y=1, xanchor="right", x=1, bgcolor='rgba(255,255,255,1)', bordercolor='Black',borderwidth=1))
fig.show()

In [ ]:
# Comparison Hmd delta_rel GRcube vs GRlog Balakhany VIII sand
fig = go.Figure()
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION == 'Balakhany VIII sand') &
                                     (gross_net_rel_fld_xy.field_num <=4)].delta_rel, 
              xbins=dict(start=0, end=1, size=0.05), marker_color='#06cfc8', name='Chirag'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION == 'Balakhany VIII sand') &
                                     (gross_net_rel_fld_xy.field_num > 4)].delta_rel, 
              xbins=dict(start=0, end=1, size=0.05), marker_color='#ff0845', name='Azeri'))
fig.update_traces(opacity=0.5)
fig.update_layout(title_text='Comparison Hmd delta_rel GRcube vs GRlog Balakhany VIII sand',xaxis_title_text='delta_rel', yaxis_title_text='Count',
                  autosize=True, width=1000, height=300, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(barmode='overlay')
fig.update_xaxes(nticks=40, showgrid=True)

In [ ]:
# Balakhany VIII sand delta_rel Hmd GRcube vs GRlog
fig = go.Figure()
field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
gross_net_rel_fld_xy_8 =gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION=='Balakhany VIII sand') & 
                                             (gross_net_rel_fld_xy.delta_rel.notna()) &
                                             (gross_net_rel_fld_xy.delta_rel <=1)]
gross_net_rel_fld_xy_10 =gross_net_rel_fld_xy[gross_net_rel_fld_xy.FORMATION=='Balakhany X sand']
color_scale = ('#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692')
fig.add_trace(go.Scatter(x=gross_net_rel_fld_xy_8.X, y=gross_net_rel_fld_xy_8.Y, customdata = gross_net_rel_fld_xy_8[['wellName', 'delta_rel','field']],
                         marker=dict(color=gross_net_rel_fld_xy_8.field_num, size=gross_net_rel_fld_xy_8.delta_rel*30, 
                                     line=dict(color='rgb(47, 57, 61)', width=1), colorscale=color_scale),
                         mode='markers', name='VSHcube vs VSHlof delta_rel',
                         hovertemplate="".join(["well:%{customdata[0]}, delta_rel:%{customdata[1]}<extra></extra>"])))
fig.add_trace(go.Scatter(x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
                         text=field_avg_coord['field'], textposition="middle right",
                         marker=dict(color='rgb(0, 0,0)', size=12),
                         mode='markers+text', name='Platforms', 
                         marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])))
fig.update_layout(title_text='Balakhany VIII sand delta_rel (v/v) Hmd GRcube vs GRlog, color=f(platform)',
                  autosize=True, width=1200, height=700, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(legend=dict( yanchor="top", y=1, xanchor="right", x=1, bgcolor='rgba(255,255,255,1)', bordercolor='Black',borderwidth=1))
fig.show()

In [ ]:
# Balakhany X sand delta_rel 3-colors Hmd GRcube vs GRlog
fig = go.Figure()
field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
gross_net_rel_fld_xy_8 =gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION=='Balakhany VIII sand') & 
                                             (gross_net_rel_fld_xy.delta_rel.notna()) &
                                             (gross_net_rel_fld_xy.delta_rel <=1)]
gross_net_rel_fld_xy_10 =gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION=='Balakhany X sand') & 
                                              (gross_net_rel_fld_xy.delta_rel.notna()) &
                                              (gross_net_rel_fld_xy.delta_rel <=1)]
color_scale = ('#02bd46', '#ffb303', '#d40404')                             
fig.add_trace(go.Scatter(x=gross_net_rel_fld_xy_10.X, y=gross_net_rel_fld_xy_10.Y, customdata = gross_net_rel_fld_xy_10[['wellName', 'delta_rel','field']],
                         marker=dict(color=gross_net_rel_fld_xy_10.qc_gross_net, size=10, 
                                     line=dict(color='rgb(47, 57, 61)', width=1), colorscale=color_scale),
                         mode='markers', name='vshc/vshl delta_rel',
                         hovertemplate="".join(["well:%{customdata[0]}, delta_rel:%{customdata[1]}<extra></extra>"])))
fig.add_trace(go.Scatter(x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
                         text=field_avg_coord['field'], textposition="middle right",
                         marker=dict(color='rgb(0, 0,0)', size=12),
                         mode='markers+text', name='Platforms', 
                         marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])))
fig.update_layout(title_text='Balakhany X sand delta_rel (v/v) Hmd GRcube vs GRlog, color=f(platform)',
                  autosize=True, width=1200, height=700, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(legend=dict( yanchor="top", y=1, xanchor="right", x=1, bgcolor='rgba(255,255,255,1)', bordercolor='Black',borderwidth=1))
fig.show()

In [211]:
# well = 'D23Z'
# well_display_vsh_gr10(well)
# well_sheet_display(well, 2)

In [ ]:
# Comparison Hmd delta_rel GRcube vs GRlog Balakhany X sand
fig = go.Figure()
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION == 'Balakhany X sand') &
                                     (gross_net_rel_fld_xy.field_num <=4)].delta_rel, 
              xbins=dict(start=0, end=1, size=0.05), marker_color='#0522fa', name='Chirag'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION == 'Balakhany X sand') &
                                     (gross_net_rel_fld_xy.field_num > 4)].delta_rel, 
              xbins=dict(start=0, end=1, size=0.05), marker_color='#f1f500', name='Azeri'))
fig.update_traces(opacity=0.5)
fig.update_layout(title_text='Comparison Hmd delta_rel GRcube vs GRlog Balakhany X sand',xaxis_title_text='delta_rel', yaxis_title_text='Count',
                  autosize=True, width=1000, height=300, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(barmode='overlay')
fig.update_xaxes(nticks=40, showgrid=True)

In [ ]:
# Balakhany X sand delta_rel platform-color Hmd GRcube vs GRlog
fig = go.Figure()
field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
gross_net_rel_fld_xy_8 =gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION=='Balakhany VIII sand') & 
                                             (gross_net_rel_fld_xy.delta_rel.notna()) &
                                             (gross_net_rel_fld_xy.delta_rel <=1)]
gross_net_rel_fld_xy_10 =gross_net_rel_fld_xy[(gross_net_rel_fld_xy.FORMATION=='Balakhany X sand') & 
                                              (gross_net_rel_fld_xy.delta_rel.notna()) &
                                              (gross_net_rel_fld_xy.delta_rel <=1)]
color_scale = ('#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692')                              
fig.add_trace(go.Scatter(x=gross_net_rel_fld_xy_10.X, y=gross_net_rel_fld_xy_10.Y, customdata = gross_net_rel_fld_xy_10[['wellName', 'delta_rel','field']],
                         marker=dict(color=gross_net_rel_fld_xy_10.field_num, size=gross_net_rel_fld_xy_10.delta_rel*30, 
                                     line=dict(color='rgb(47, 57, 61)', width=1), colorscale=color_scale),
                         mode='markers', name='VSHcube vs VSHlof delta_rel',
                         hovertemplate="".join(["well:%{customdata[0]}, delta_rel:%{customdata[1]}<extra></extra>"])))
fig.add_trace(go.Scatter(x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
                         text=field_avg_coord['field'], textposition="middle right",
                         marker=dict(color='rgb(0, 0,0)', size=12),
                         mode='markers+text', name='Platforms', 
                         marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])))
fig.update_layout(title_text='Balakhany X sand delta_rel (v/v) Hmd GRcube vs GRlog, color=f(platform)',
                  autosize=True, width=1200, height=700, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(legend=dict( yanchor="top", y=1, xanchor="right", x=1, bgcolor='rgba(255,255,255,1)', bordercolor='Black',borderwidth=1))
fig.show()

#### Checking influence of TST-thick to GRcubeVSH

In [214]:
# Loading the TL well-logs csv file
path = 'C:\\jupyter\\SPP\\input\\'
df_prq = pd.read_parquet(path + 'ACG_wells_JOINT_BEST_v6.parquet.gzip')
# Calculation tst-thickness of each formation
df_prq_prep = df_prq[['wellName', 'DEPTH', 'FORMATION','Fluidcode_mod', 'TVD_SCS', 'TST',]]
df_prq_prep = df_prq_prep[(df_prq_prep.FORMATION == 'Balakhany VIII sand') | (df_prq_prep.FORMATION == 'Balakhany X sand')]
df_prq_thick_tst = df_prq_prep.groupby(['wellName','FORMATION'])['TST'].apply(lambda x: x.iloc[-1] - x.iloc[0]).reset_index()
df_prq_thick_tst['TST'] = df_prq_thick_tst.TST.round(0) 
df_prq_thick_tst = df_prq_thick_tst[df_prq_thick_tst.TST > 0]
df_prq_thick_tst.rename(columns={'TST':'thickn_tst'}, inplace=True)
gross_net_rel_fld_xy_8_tst = gross_net_rel_fld_xy_8.set_index(['wellName', 'FORMATION']).join(df_prq_thick_tst.set_index(['wellName', 'FORMATION'])).reset_index()
gross_net_rel_fld_xy_10_tst = gross_net_rel_fld_xy_10.set_index(['wellName', 'FORMATION']).join(df_prq_thick_tst.set_index(['wellName', 'FORMATION'])).reset_index()

In [ ]:
# Distribution Hmd-delta_rel quality over tst-thickness Balaknany VIII sand Azeri zone
fig = go.Figure()
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_8_tst[(gross_net_rel_fld_xy_8_tst.qc_gross_net_v2 == 4) &
                                           (gross_net_rel_fld_xy_8_tst.field_num > 4)].thickn_tst, 
              xbins=dict(start=0, end=100, size=1), marker_color='grey', name='grey'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_8_tst[(gross_net_rel_fld_xy_8_tst.qc_gross_net_v2 == 3) &
                                           (gross_net_rel_fld_xy_8_tst.field_num > 4)].thickn_tst, 
              xbins=dict(start=0, end=100, size=1), marker_color='red', name='red'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_8_tst[(gross_net_rel_fld_xy_8_tst.qc_gross_net_v2 == 2) &
                                           (gross_net_rel_fld_xy_8_tst.field_num > 4)].thickn_tst, 
              xbins=dict(start=0, end=100, size=1), marker_color='yellow', name='yellow'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_8_tst[(gross_net_rel_fld_xy_8_tst.qc_gross_net_v2 == 1) &
                                           (gross_net_rel_fld_xy_8_tst.field_num > 4)].thickn_tst, 
              xbins=dict(start=0, end=100, size=1), marker_color='green', name='green'))
fig.update_traces(opacity=0.75)
fig.update_layout(title_text='Distribution Hmd-delta_rel quality over tst-thickness Balaknany VIII sand Azeri zone',
                  xaxis_title_text='tst_thickness', yaxis_title_text='Count',
                  autosize=True, width=1000, height=300, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(barmode='stack')
fig.update_xaxes(nticks=40, showgrid=True)

In [ ]:
# Distribution Hmd-delta_rel quality over tst-thickness Balaknany VIII sand Chirag zone
fig = go.Figure()
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_8_tst[(gross_net_rel_fld_xy_8_tst.qc_gross_net_v2 == 4) &
                                           (gross_net_rel_fld_xy_8_tst.field_num <= 4)].thickn_tst, 
              xbins=dict(start=50, end=200, size=5), marker_color='grey', name='grey'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_8_tst[(gross_net_rel_fld_xy_8_tst.qc_gross_net_v2 == 3) &
                                           (gross_net_rel_fld_xy_8_tst.field_num <= 4)].thickn_tst, 
              xbins=dict(start=50, end=200, size=5), marker_color='red', name='red'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_8_tst[(gross_net_rel_fld_xy_8_tst.qc_gross_net_v2 == 2) &
                                           (gross_net_rel_fld_xy_8_tst.field_num <= 4)].thickn_tst, 
              xbins=dict(start=50, end=200, size=5), marker_color='yellow', name='yellow'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_8_tst[(gross_net_rel_fld_xy_8_tst.qc_gross_net_v2 == 1) &
                                           (gross_net_rel_fld_xy_8_tst.field_num <= 4)].thickn_tst, 
              xbins=dict(start=50, end=200, size=5), marker_color='green', name='green'))
fig.update_traces(opacity=0.75)
fig.update_layout(title_text='Distribution Hmd-delta_rel quality over tst-thickness Balaknany VIII sand Chirag zone',
                  xaxis_title_text='tst_thickness', yaxis_title_text='Count',
                  autosize=True, width=1000, height=300, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(barmode='stack')
fig.update_xaxes(nticks=40, showgrid=True)

In [ ]:
# Distribution Hmd-delta_rel quality over tst-thickness Balaknany X sand Chirag zone
fig = go.Figure()
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_10_tst[(gross_net_rel_fld_xy_10_tst.qc_gross_net_v2 == 4) &
                                           (gross_net_rel_fld_xy_10_tst.field_num <= 7)].thickn_tst, 
              xbins=dict(start=0, end=200, size=2), marker_color='grey', name='grey'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_10_tst[(gross_net_rel_fld_xy_10_tst.qc_gross_net_v2 == 3) &
                                           (gross_net_rel_fld_xy_10_tst.field_num <= 7)].thickn_tst, 
              xbins=dict(start=0, end=200, size=2), marker_color='red', name='red'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_10_tst[(gross_net_rel_fld_xy_10_tst.qc_gross_net_v2 == 2) &
                                           (gross_net_rel_fld_xy_10_tst.field_num <= 7)].thickn_tst, 
              xbins=dict(start=0, end=200, size=2), marker_color='yellow', name='yellow'))
fig.add_trace(go.Histogram(
              x=gross_net_rel_fld_xy_10_tst[(gross_net_rel_fld_xy_10_tst.qc_gross_net_v2 == 1) &
                                           (gross_net_rel_fld_xy_10_tst.field_num <= 7)].thickn_tst, 
              xbins=dict(start=0, end=200, size=2), marker_color='green', name='green'))
fig.update_traces(opacity=0.75)
fig.update_layout(title_text='Distribution Hmd-delta_rel quality over tst-thickness Balaknany X sand',
                  xaxis_title_text='tst_thickness', yaxis_title_text='Count',
                  autosize=True, width=1000, height=300, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(barmode='stack')
fig.update_xaxes(nticks=40, showgrid=True)

In [ ]:
# Balakhany VIII sand delta_rel 4-colors_v2 Hmd GRcube vs GRlog with tst-thickness FU
fig = fig = make_subplots(rows=2, cols=1, subplot_titles=("Chirag", "Azeri"), 
                          vertical_spacing = 0.025)
# field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
gross_net_rel_fld_xy_8_tst_chi =gross_net_rel_fld_xy_8_tst[(gross_net_rel_fld_xy_8_tst.FORMATION=='Balakhany VIII sand') & 
                                                           (gross_net_rel_fld_xy_8_tst.delta_rel.notna()) &
                                                           (gross_net_rel_fld_xy_8_tst.delta_rel <=1) &
                                                           (gross_net_rel_fld_xy_8_tst.thickn_tst.notna()) &
                                                           (gross_net_rel_fld_xy_8_tst.field_num <=4)] 
gross_net_rel_fld_xy_8_tst_aze =gross_net_rel_fld_xy_8_tst[(gross_net_rel_fld_xy_8_tst.FORMATION=='Balakhany VIII sand') & 
                                                           (gross_net_rel_fld_xy_8_tst.delta_rel.notna()) &
                                                           (gross_net_rel_fld_xy_8_tst.delta_rel <=1) &
                                                           (gross_net_rel_fld_xy_8_tst.thickn_tst.notna()) &
                                                           (gross_net_rel_fld_xy_8_tst.field_num > 4)]  
# gross_net_rel_fld_xy_10 =gross_net_rel_fld_xy[gross_net_rel_fld_xy.FORMATION=='Balakhany X sand']
color_scale = ('#02bd46', '#ffb303', '#d40404', '#827d7d')
fig.add_trace(go.Scatter(x=gross_net_rel_fld_xy_8_tst_chi.X, y=gross_net_rel_fld_xy_8_tst_chi.Y, 
                         customdata = gross_net_rel_fld_xy_8_tst_chi[['wellName', 'delta_rel','thickn_tst']],
                         marker=dict(color=gross_net_rel_fld_xy_8_tst_chi.qc_gross_net_v2, size=gross_net_rel_fld_xy_8_tst_chi.thickn_tst*0.1, 
                                     line=dict(color='rgb(47, 57, 61)', width=1), colorscale=color_scale),
                         mode='markers', name='vshc/vshl delta_rel',
                         hovertemplate="".join(["well:%{customdata[0]}, delta_rel:%{customdata[1]}, tst:%{customdata[2]}<extra></extra>"])),
                         row=1, col=1)
fig.add_trace(go.Scatter(x=gross_net_rel_fld_xy_8_tst_aze.X, y=gross_net_rel_fld_xy_8_tst_aze.Y, 
                         customdata = gross_net_rel_fld_xy_8_tst_aze[['wellName', 'delta_rel','thickn_tst']],
                         marker=dict(color=gross_net_rel_fld_xy_8_tst_aze.qc_gross_net_v2, size=gross_net_rel_fld_xy_8_tst_aze.thickn_tst, 
                                     line=dict(color='rgb(47, 57, 61)', width=1), colorscale=color_scale),
                         mode='markers', name='vshc/vshl delta_rel',
                         hovertemplate="".join(["well:%{customdata[0]}, delta_rel:%{customdata[1]}, tst:%{customdata[2]}<extra></extra>"])),
                         row=2, col=1)
fig.update_layout(title_text='Balakhany X sand delta_rel Hmd GRcube vs GRlog, color=f(qc), size=f(TST)',
                  autosize=True, width=1200, height=1400, margin=dict(l=10,r=10,b=10,t=60))
fig.update_layout(legend=dict( yanchor="top", y=1, xanchor="right", x=1, bgcolor='rgba(255,255,255,1)', bordercolor='Black',borderwidth=1))
fig.show()

In [ ]:
# Balakhany X sand delta_rel 4-colors_v2 Hmd GRcube vs GRlog with tst-thickness FU
fig = go.Figure()
field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
gross_net_rel_fld_xy_10_tst =gross_net_rel_fld_xy_10_tst[(gross_net_rel_fld_xy_10_tst.FORMATION=='Balakhany X sand') & 
                                                       (gross_net_rel_fld_xy_10_tst.delta_rel.notna()) &
                                                       (gross_net_rel_fld_xy_10_tst.delta_rel <=1) &
                                                       (gross_net_rel_fld_xy_10_tst.thickn_tst.notna())]  
# gross_net_rel_fld_xy_10 =gross_net_rel_fld_xy[gross_net_rel_fld_xy.FORMATION=='Balakhany X sand']
color_scale = ('#02bd46', '#ffb303', '#d40404', '#827d7d')
fig.add_trace(go.Scatter(x=gross_net_rel_fld_xy_10_tst.X, y=gross_net_rel_fld_xy_10_tst.Y, customdata = gross_net_rel_fld_xy_10_tst[['wellName', 'delta_rel','thickn_tst']],
                         marker=dict(color=gross_net_rel_fld_xy_10_tst.qc_gross_net_v2, size=gross_net_rel_fld_xy_10_tst.thickn_tst*0.3, 
                                     line=dict(color='rgb(47, 57, 61)', width=1), colorscale=color_scale),
                         mode='markers', name='wells',
                         hovertemplate="".join(["well:%{customdata[0]}, delta_rel:%{customdata[1]}, tst:%{customdata[2]}<extra></extra>"])))
fig.add_trace(go.Scatter(x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
                         text=field_avg_coord['field'], textposition="middle right",
                         marker=dict(color='rgb(0, 0,0)', size=12),
                         mode='markers+text', name='Platforms', 
                         marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])))
fig.update_layout(title_text='Balakhany X sand delta_rel Hmd GRcube vs GRlog, color=f(qc), size=f(tst_thickness)',
                  autosize=True, width=1200, height=700, margin=dict(l=10,r=10,b=10,t=40))
fig.update_layout(legend=dict( yanchor="top", y=1, xanchor="right", x=1, bgcolor='rgba(255,255,255,1)', bordercolor='Black',borderwidth=1))
fig.show()

In [ ]:
# Overpaling of seismic quality polygon and delta_rel_vsh for Balakhany VIII sand
gdf = gpd.read_file("input\polygons_geojson\K10_tbalxSand_Poor_Data_Quality_ZACI_5174_SCF_M400000_QLSKPrSDM_abkk1l.ptd.geojson")
gdf.plot(aspect='equal', figsize=(14,9), alpha=0.25, color='blue', label='seismic poor', ec='black')
green = gross_net_rel_fld_xy_8_tst[gross_net_rel_fld_xy_8_tst.qc_gross_net==1]
yellow = gross_net_rel_fld_xy_8_tst[gross_net_rel_fld_xy_8_tst.qc_gross_net==2]
red = gross_net_rel_fld_xy_8_tst[gross_net_rel_fld_xy_8_tst.qc_gross_net==3]
plt.scatter(green.X, green.Y, c='green',alpha=0.75, ec='black', s=50)
plt.scatter(yellow.X, yellow.Y, c='orange',alpha=0.75, ec='black', s=50)
plt.scatter(red.X, red.Y, c='red',alpha=0.75, ec='black', s=50)
plt.title('Balakhany VIII sand delta_rel_vsh Hmd GRcube vs GRlog, color=f(qc), seismic=poor_quality')
plt.show()

In [ ]:
# Overpaling of seismic quality polygon and delta_rel_vsh for Balakhany X sand
gdf = gpd.read_file("input\polygons_geojson\K10_tbalxSand_Poor_Data_Quality_ZACI_5174_SCF_M400000_QLSKPrSDM_abkk1l.ptd.geojson")
gdf.plot(aspect='equal', figsize=(14,9), alpha=0.25, color='blue', label='seismic poor', ec='black')
green = gross_net_rel_fld_xy_10_tst[gross_net_rel_fld_xy_10_tst.qc_gross_net==1]
yellow = gross_net_rel_fld_xy_10_tst[gross_net_rel_fld_xy_10_tst.qc_gross_net==2]
red = gross_net_rel_fld_xy_10_tst[gross_net_rel_fld_xy_10_tst.qc_gross_net==3]
plt.scatter(green.X, green.Y, c='green',alpha=0.75, ec='black', s=40)
plt.scatter(yellow.X, yellow.Y, c='orange',alpha=0.75, ec='black', s=40)
plt.scatter(red.X, red.Y, c='red',alpha=0.75, ec='black', s=40)
plt.title('Balakhany X sand delta_rel_vsh Hmd GRcube vs GRlog, color=f(qc), seismic=poor_quality')
plt.show()

### Comparison AUC and VSH_delta_rel

In [ ]:
data_auc8.head(3)

In [ ]:
data_vsh_drel.rename(columns={'delta':'delta_vsh', 'delta_rel':'delta_rel_vsh'}, inplace=True)
data_vsh_drel8 = data_vsh_drel[data_vsh_drel.FORMATION == 'Balakhany VIII sand']
data_vsh_drel8.head(3)

In [ ]:
df_comparison = data_auc8[['wellName', 'delta_rel_auc']].set_index('wellName').join(
                data_vsh_drel8[['wellName','delta_rel_vsh', 'qc_gross_net']].set_index('wellName')).reset_index()
df_comparison['delta_rel_auc'] = (df_comparison.delta_rel_auc/100).round(3)
df_comparison.head(3)

In [ ]:
fig1 = go.Figure()
color_scale = ('#02bd46', '#ffb303', '#d40404')   
fig1.add_trace(go.Scatter(x=df_comparison.delta_rel_auc, y=df_comparison.delta_rel_vsh, customdata = df_comparison[['wellName', 'delta_rel_auc','delta_rel_vsh']],
                         marker=dict(color=df_comparison.qc_gross_net, size=10, 
                                     line=dict(color='rgb(47, 57, 61)', width=1), colorscale=color_scale),
                         mode='markers', name='vshc/vshl delta_rel',
                         hovertemplate="".join(["well:%{customdata[0]}, delta_rel_auc:%{customdata[1]}, delta_rel_vsh:%{customdata[2]}<extra></extra>"])))
fig1.update_layout(xaxis_range=[-0.05,0.5], yaxis_range=[-0.2,3.3])
fig1.update_layout(title_text='Balakhany VIII sand delta_rel_auc vs delta_rel_vsh', xaxis_title="delta_rel_auc", yaxis_title="delta_rel_vsh",
                  autosize=True, width=700, height=400, margin=dict(l=10,r=10,b=10,t=40))

In [ ]:
well='F02'
well_display_auc(well)
well_display_vsh_gr8(well)
well_sheet_display(well,1)

### Checking idea to delete by smaller Hmd instead of strickly Hmd_cb or Hmd_lg

In [ ]:
# Testing script for smaller divider
gr_testdiv = gr_cube_gross_net_total2.copy()
gr_testdiv['gross_net_lg_lower'] = gr_testdiv.gross_net_lg
gr_testdiv.loc[gr_testdiv['gross_net_cb'] < gr_testdiv['gross_net_lg'], 'gross_net_lg_lower'] = gr_testdiv['gross_net_cb']
# Main script
gr_testdiv['delta'] = abs(gr_testdiv.gross_net_lg - gr_testdiv.gross_net_cb)
gr_testdiv['delta_rel'] = (gr_testdiv.delta/gr_testdiv.gross_net_lg_lower).round(2)
# Assigning XY data
gr_testdiv_xy = gr_testdiv.set_index(['wellName', 'FORMATION']).join(df_prq_fm).reset_index()
# Assigning categorical marks to qc_gross_net_v2
gr_testdiv_xy.loc[  gr_testdiv_xy.delta_rel <=0.25, 'qc_gross_net_v2'] = 1
gr_testdiv_xy.loc[  (gr_testdiv_xy.delta_rel >0.25) &
                    (gr_testdiv_xy.delta_rel <=0.50), 'qc_gross_net_v2'] = 2
gr_testdiv_xy.loc[  (gr_testdiv_xy.delta_rel >0.50) &
                    (gr_testdiv_xy.delta_rel <=0.75), 'qc_gross_net_v2'] = 3
gr_testdiv_xy.loc[  (gr_testdiv_xy.delta_rel >0.75),'qc_gross_net_v2'] = 4
# Balakhany VIII sand delta_rel 3-colors Hmd GRcube vs GRlog smaller divider
fig_test = go.Figure()
field_avg_coord = metadata.groupby('field')[['X_wellhead','Y_wellhead']].mean().reset_index()
gr_testdiv_8 =gr_testdiv_xy[(gr_testdiv_xy.FORMATION=='Balakhany VIII sand') & 
                            (gr_testdiv_xy.delta_rel.notna()) &
                            (gr_testdiv_xy.delta_rel <=1)]
# gross_net_rel_fld_xy_10 =gross_net_rel_fld_xy[gross_net_rel_fld_xy.FORMATION=='Balakhany X sand']
color_scale = ('#02bd46', '#ffb303', '#d40404', '#827d7d')
fig_test.add_trace(go.Scatter(x=gr_testdiv_8.X, y=gr_testdiv_8.Y, customdata = gr_testdiv_8[['wellName', 'delta_rel']],
                         marker=dict(color=gr_testdiv_8.qc_gross_net_v2, size=10, 
                                     line=dict(color='rgb(47, 57, 61)', width=1), colorscale=color_scale),
                         mode='markers', name='vshc/vshl delta_rel',
                         hovertemplate="".join(["well:%{customdata[0]}, delta_rel:%{customdata[1]}<extra></extra>"])))
fig_test.add_trace(go.Scatter(x=field_avg_coord.X_wellhead, y=field_avg_coord.Y_wellhead, customdata = field_avg_coord[['field']],
                         text=field_avg_coord['field'], textposition="middle right",
                         marker=dict(color='rgb(0, 0,0)', size=12),
                         mode='markers+text', name='Platforms', 
                         marker_symbol='square', hovertemplate="".join(["%{customdata[0]}<extra></extra>"])))
fig_test.update_layout(title_text='Balakhany VIII sand delta_rel (v/v) Hmd GRcube vs GRlog, color=f(platform)',
                  autosize=True, width=1200, height=700, margin=dict(l=10,r=10,b=10,t=40))
fig_test.update_layout(legend=dict( yanchor="top", y=1, xanchor="right", x=1, bgcolor='rgba(255,255,255,1)', bordercolor='Black',borderwidth=1))
fig_test.show()

In [ ]:
well_display_vsh_gr8('A05')
well_sheet_display('A05',1)

In [ ]:
gr_testdiv_xy[(gr_testdiv_xy.qc_gross_net_v2>=3) & (gr_testdiv_xy.FORMATION=='Balakhany VIII sand')]

In [ ]:
for wells in gr_testdiv_xy[(gr_testdiv_xy.qc_gross_net_v2>=3) & (gr_testdiv_xy.FORMATION=='Balakhany VIII sand')]['wellName'].unique():
    well_display_vsh_gr8(wells)
    well_sheet_display(wells,1)